## Fichas da Formação - Museu de Antropologia
- Extração de dados de documentos .docx

# 1 - Primeira Etapa
Para a estração foi feito uso de um script em python que identifica os documentos .docx e a partir dai extrai suas informações,
uma vez que um documento do Word pode ser compreendido como um arquivo XML onde as informações estão comprimidas, o que o 
script faz é descompactar esses dados e gerar uma lista onde é feito o armazenamento e posteriormante é gerado um arquivo CSV,
com todos os dados.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jan 25 22:21:08 2019

@author: eduardo
"""
import os
teste = []

try:
    from xml.etree.cElementTree import XML
except ImportError:
    from xml.etree.ElementTree import XML
import zipfile


"""
Module that extract text from MS XML Word document (.docx).
(Inspired by python-docx <https://github.com/mikemaccana/python-docx>)
"""

WORD_NAMESPACE = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'
PARA = WORD_NAMESPACE + 'p'
TEXT = WORD_NAMESPACE + 't'


def get_docx_text(path):
    """
    Take the path of a docx file as argument, return the text in unicode.
    """
    document = zipfile.ZipFile(path)
    xml_content = document.read('word/document.xml')
    document.close()
    tree = XML(xml_content)

    paragraphs = []
    for paragraph in tree.getiterator(PARA):
        texts = [node.text
                 for node in paragraph.getiterator(TEXT)
                 if node.text]
        if texts:
            paragraphs.append(''.join(texts))

    return '\n'.join(paragraphs)


for root, dirs, files in os.walk(os.path.abspath("C:\\Users\\eduar\\Desktop\\fichas ufg\\")):
    for file in files:
        teste.append(get_docx_text(os.path.join(root, file)))
        
import csv

with open('C:\\Users\\eduar\\Desktop\\fichas ufg\\itens.csv', 'w', encoding="utf-8") as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_MINIMAL)
     #wr.writerows(teste)
     for val in teste:
        wr.writerow([val])    
    

Link de referência para o código: http://etienned.github.io/posts/extract-text-from-word-docx-simply/

# 2 - Segunda Etapa
Uma vez que temos todo o texto de cada um dos arquivos em uma linha do arquivo CSV gerado, passamos ao tratamento, aqui foi feito
o uso do Notepad++ o qual facilita a visualização te todo o conteúdo do CSV, a partir desse ponto, foram retirado espaços indesejados,
verificação e correção de textos ou nome de metadados repetidos de forma diferente, como "Nº. item" e "No. item", além da retirada do
delimitador de conteúdo do CSV, normalmente as aspas "", isso faz com que ao invés de todo o texto ficar numa linha, cada linha do texto
se torne uma linha no CSV de acordo com a quebra de linha. 

Feito isso é preciso abrir os dados novamente no excel e efetuar verificações para achar repetições, uma vez que todo os metadados tem
seus nome apresentados antes de ':', fez-se o uso desses ':' para separar os dados dentro do arquivo CSV (normalmente se usar ; ou ,).

Esse processo se repete até todos os dados estarem organizados e por fim, mais uma vez Notepad++ o delimitador de texto ':' é alterado
para outro simbolo de forma que não afete o texto.
 

# 3 - Terceira Etapa
Após o tratamento, são obtidas duas colunas (Metadados; Conteúdo), os metadados se repetem para cada um dos itens, ou seja, na coluna metadados temos várias linhas com o nome "Descrição" etc, para contornar isso, podemos utilizar uma função de indice do Excel, onde todos os dados serão agrupados de acordo com o texto da coluna Metadados.

Nesse tratamento foi usado uma Macro, o que facilita o processo.

In [ ]:
Como utilizar:
    1 - Com o excel aberto aperte ALT+F11, isso irá abrir o Microsoft Visual Basic for Applications
    2 - Clique em Inserir>Modulo, e cole o Código Abaixo na janela Modulo.

Sub transposeunique()
'updateby Extendoffice 20151207
    Dim xLRow As Long
    Dim i As Long
    Dim xCrit As String
    Dim xCol  As New Collection
    Dim xRg As Range
    Dim xOutRg As Range
    Dim xTxt As String
    Dim xCount As Long
    Dim xVRg As Range
    On Error Resume Next
    xTxt = ActiveWindow.RangeSelection.Address
    Set xRg = Application.InputBox("please select data range(only two columns):", "Kutools for Excel", xTxt, , , , , 8)
    Set xRg = Application.Intersect(xRg, xRg.Worksheet.UsedRange)
    If xRg Is Nothing Then Exit Sub
    If (xRg.Columns.Count <> 2) Or _
       (xRg.Areas.Count > 1) Then
        MsgBox "the used range is only one area with two columns ", , "Kutools for Excel"
        Exit Sub
    End If
    Set xOutRg = Application.InputBox("please select output range(specify one cell):", "Kutools for Excel", xTxt, , , , , 8)
    If xOutRg Is Nothing Then Exit Sub
    Set xOutRg = xOutRg.Range(1)
    xLRow = xRg.Rows.Count
    For i = 2 To xLRow
        xCol.Add xRg.Cells(i, 1).Value, xRg.Cells(i, 1).Value
    Next
    Application.ScreenUpdating = False
    For i = 1 To xCol.Count
        xCrit = xCol.Item(i)
        xOutRg.Offset(i, 0) = xCrit
        xRg.AutoFilter Field:=1, Criteria1:=xCrit
        Set xVRg = xRg.Range("B2:B" & xLRow).SpecialCells(xlCellTypeVisible)
        If xVRg.Count > xCount Then xCount = xVRg.Count
        xRg.Range("B2:B" & xLRow).SpecialCells(xlCellTypeVisible).Copy
        xOutRg.Offset(i, 1).PasteSpecial Paste:=xlPasteAll, Operation:=xlNone, SkipBlanks:=False, Transpose:=True
        Application.CutCopyMode = False
    Next
    xOutRg = xRg.Cells(1, 1)
    xOutRg.Offset(0, 1).Resize(1, xCount) = xRg.Cells(1, 2)
    xRg.Rows(1).Copy
    xOutRg.Resize(1, xCount + 1).PasteSpecial Paste:=xlPasteFormats
    xRg.AutoFilter
    Application.ScreenUpdating = True
End Sub

    3 - Pressione F5 para executar, uma pequena janela de seleção será aberta, selecione as colunas com os dados, nesse caso (Metadados e Conteúdo)
    4 - Clique em OK e selecione a celula onde os novos dados irão aparece, clique em OK.

# 4 - Quarta Etapa
Nessa etapa é feita a transformação dos dados, assim que terminar a etapa 3, terá uma tabela com diversas colunas e todos os dados,
com base nos metadados, para transformá-los, selecione todas as linhas e colunas geradas pela etapa anterior.

1. Vá até a aba Dados>Da tabela/intervalo, isso irá abrir uma nova janela do excel.
2. Vá ná aba Transformar>Transpor, isso irá tirar os metadados das linhas e coloca-los como colunas.
3. Volte para aba Base>Fechar & Carregar.

feito isso os dados estão prontos, para serem revisados e inseridos no Tainacan.